In [2]:
from datetime import datetime, timedelta
from tabulate import tabulate

class Driver:
    def __init__(self, identifier, shift_start, total_hours):
        self.identifier = identifier
        self.start_time = datetime.strptime(shift_start, '%H:%M')
        self.total_hours = total_hours
        self.log = []
        self.route_number = 1

    def log_event(self, timestamp, event):
        self.log.append(f"{timestamp.strftime('%H:%M')} - {event}")

class ScheduleManager:
    def __init__(self, driver_list, break_long=60, break_short=15):
        self.driver_list = driver_list
        self.break_long_duration = timedelta(minutes=break_long)
        self.break_short_duration = timedelta(minutes=break_short)
        self.peak_intervals = [
            (datetime.strptime("07:00", "%H:%M").time(), datetime.strptime("09:00", "%H:%M").time()),
            (datetime.strptime("17:00", "%H:%M").time(), datetime.strptime("19:00", "%H:%M").time())
        ]

    def is_within_peak(self, timestamp):
        return any(start <= timestamp.time() < end for start, end in self.peak_intervals)

    def construct_schedule(self):
        for driver in self.driver_list:
            current_time = driver.start_time
            hours_completed = 0

            driver.log_event(current_time, 'Смена началась')
            current_time += timedelta(hours=1)
            hours_completed += 1

            rest_types = ['long'] if driver.total_hours > 8 else ['short', 'short']
            rest_timings = [4] if driver.total_hours > 8 else [3, 6]

            while hours_completed < driver.total_hours:
                if rest_timings and hours_completed == rest_timings[0]:
                    if not rest_types:
                        rest_timings.pop(0)
                        continue
                    current_rest = rest_types.pop(0)
                    if self.is_within_peak(current_time):
                        driver.log_event(current_time, 'Перерыв перенесён (час пик)')
                        current_time += timedelta(hours=1)
                        rest_timings[0] += 1
                        continue
                    if current_rest == 'long':
                        driver.log_event(current_time, 'Длинный перерыв (60 минут)')
                        current_time += self.break_long_duration
                    else:
                        driver.log_event(current_time, 'Короткий перерыв (15 минут)')
                        current_time += self.break_short_duration
                    rest_timings.pop(0)
                    continue

                task = f'Поездка по маршруту {driver.route_number}'
                if self.is_within_peak(current_time):
                    task += ' (час пик)'
                driver.log_event(current_time, task)
                current_time += timedelta(hours=1)
                hours_completed += 1

                driver.route_number = 2 if driver.route_number == 1 else 1

            if driver.identifier == "Водитель 8" and current_time.time() > datetime.strptime("03:00", "%H:%M").time():
                current_time = current_time.replace(hour=3, minute=0, second=0)
            driver.log_event(current_time, 'Смена завершена')

    def render_schedule(self):
        column_headers = [driver.identifier for driver in self.driver_list]
        schedule_matrix = []

        max_events = max(len(driver.log) for driver in self.driver_list)
        for row_index in range(max_events):
            row_data = []
            for driver in self.driver_list:
                row_data.append(driver.log[row_index] if row_index < len(driver.log) else '')
            schedule_matrix.append(row_data)

        print(tabulate(schedule_matrix, headers=column_headers, tablefmt='grid', stralign='left'))



driver_roster = [
    Driver("Водитель 1", "06:00", 8),
    Driver("Водитель 2", "07:00", 8),
    Driver("Водитель 3", "08:00", 8),
    Driver("Водитель 4", "11:00", 8),
    Driver("Водитель 5", "13:00", 8),
    Driver("Водитель 6", "15:00", 8),
    Driver("Водитель 7", "17:00", 8),
    Driver("Водитель 8", "18:00", 9)
]

schedule_manager = ScheduleManager(driver_roster)
schedule_manager.construct_schedule()
schedule_manager.render_schedule()


+-----------------------------------------+-----------------------------------------+-------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+-----------------------------------------+------------------------------------+
| Водитель 1                              | Водитель 2                              | Водитель 3                          | Водитель 4                              | Водитель 5                              | Водитель 6                              | Водитель 7                              | Водитель 8                         |
+=========================================+=========================================+=====================================+=========================================+=========================================+=========================================+=========================================+====================================+
| 06:00 - Сме